In [17]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import seaborn as sns

In [18]:
deaths_summed = pd.read_csv('total_deaths_perstate.csv',index_col=[0])
deaths_summed = deaths_summed[['State','Deaths']]

deaths_bydate = pd.read_csv('all_deaths_total.csv', index_col=[0]) 
deaths_bydate.columns = deaths_bydate.columns.str.capitalize()
deaths_bydate = deaths_bydate[['State','Deaths']]

macro =  pd.read_csv('macro_df.csv', index_col=[0])
macro = macro[['State','Percent Hispanic Or Latino','Percent White Not Hispanic Or Latino','Median Household Income']]

health = pd.read_csv('health.csv',index_col=[0])
health.rename(index={'State Name':'State'},inplace=True)
print(health.shape)

pop = pd.read_csv('state_pop_2021.csv', index_col=[0])
pop.columns = pop.columns.str.capitalize()

#print(deaths_summed.head())
#print(deaths_bydate.head())
#print(macro.head())
#print(health.head())
#print(pop.head())

(50, 236)


In [4]:
#Create final table with all determined features 
from functools import reduce

features = reduce(lambda left, right:     
                     pd.merge(left , right,
                              on = ['State']),
                     [deaths_summed, health, macro, pop])

print(features.head())

        State  Deaths  Access to Care - AnnualRank  \
0     Alabama   20505                         45.0   
1      Alaska    1350                         20.0   
2     Arizona   31455                         42.0   
3    Arkansas   12371                         43.0   
4  California   96460                         18.0   

   Adverse Childhood ExperiencesRank  Air PollutionRank  \
0                               30.0               33.0   
1                               44.0               16.0   
2                               35.0               46.0   
3                               48.0               23.0   
4                                9.0               50.0   

   Air and Water Quality - AnnualRank  All Determinants - AnnualRank  \
0                                29.0                           45.0   
1                                43.0                           31.0   
2                                41.0                           32.0   
3                               

In [15]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler() #gets data in range -1 to 1

scaler.fit(features.drop('State', axis=1))
scaled_features = scaler.transform(features.drop('State', axis=1))
scaled_data = pd.DataFrame(scaled_features, columns = features.drop('State', axis=1).columns)

In [14]:
#remove nan and infinity values 
scaled_data = scaled_data.replace([np.inf, -np.inf], np.nan)
scaled_data = scaled_data.dropna()
scaled_data = scaled_data.reset_index()
print(np.any(np.isnan(scaled_data)))
print(np.all(np.isfinite(scaled_data)))

print(scaled_data.dtypes)
print(scaled_data.shape)

False
True
index                                     int64
Deaths                                  float64
Access to Care - AnnualRank             float64
Adverse Childhood ExperiencesRank       float64
Air PollutionRank                       float64
                                         ...   
Percent Hispanic Or Latino              float64
Percent White Not Hispanic Or Latino    float64
Median Household Income                 float64
Population                              float64
% population affected                   float64
Length: 243, dtype: object
(50, 243)


In [7]:
scaled_data.head()

,index,Deaths,Access to Care - AnnualRank,Adverse Childhood ExperiencesRank,Air PollutionRank,Air and Water Quality - AnnualRank,All Determinants - AnnualRank,ArthritisRank,AsthmaRank,Avoided Care Due to CostRank,...,Voter Participation (Presidential) - Ages 65+ Rank,Voter Participation (Presidential) - FemaleRank,Voter Participation (Presidential) - MaleRank,Voter Participation (Presidential) - WhiteRank,Water FluoridationRank,Percent Hispanic Or Latino,Percent White Not Hispanic Or Latino,Median Household Income,Population,% population affected
0,0,-0.025262,1.350842,0.329091,0.556115,0.246624,1.351619,1.627838,-0.617030,1.373194,...,1.636854,1.504476,1.432738,1.435635,-0.169996,-0.740789,-0.190710,-1.187664,-0.215567,1.986908
1,1,-0.877749,-0.379232,1.292956,-0.631568,1.216496,0.382220,-1.188253,-0.826429,0.408098,...,0.665015,0.879924,0.739925,-0.161058,1.212084,-0.479880,-0.517751,1.473711,-0.801516,-0.752823
2,2,0.462064,1.143233,0.673328,1.464344,1.077943,0.451463,-0.913512,0.080968,1.028517,...,-1.417496,-1.410100,-0.645702,-1.341222,0.797460,1.877967,-0.908917,-0.361974,0.088676,1.538279
3,3,-0.387263,1.212436,1.568345,-0.142522,-0.446141,1.559347,1.215727,-0.477430,1.304258,...,1.706271,1.643266,1.709863,1.643899,-0.377308,-0.431563,0.238932,-1.476248,-0.489549,0.902111
4,4,3.355091,-0.517638,-1.116706,1.743799,1.424326,0.174492,-1.669049,-0.337831,-0.832740,...,1.081518,0.532951,0.254955,-1.132957,0.728356,2.622042,-2.037529,1.237561,4.436698,-0.202258


In [ ]:
# refer to https://medium.com/@evgen.ryzhkov/5-stages-of-data-preprocessing-for-k-means-clustering-b755426f9932

In [16]:
scaled_data.to_csv('scaled_data.csv')

In [ ]:
# Does my data set have any categorical data, such as Gender or day of the week?
# Do my features have data values that range from 0 - 100 or 0-1 or both and more?